<a href="https://colab.research.google.com/github/Andrew-TraverseMT/NYC_Addresses/blob/main/extract_address_from_taxbill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook takes records from MapPluto and looks up parcel owner mailing addresses from property tax assessments using the BBL.

In [1]:
!pip install pdfplumber
!pip install PyPDF2
!pip install pymupdf

In [2]:
import requests
import pdfplumber
import pymupdf
import io
import PyPDF2
import re
import pandas as pd
from time import sleep
import warnings
import logging
import time
from tqdm import tqdm
import sys

In [3]:
def extract_all_text_from_url(url):
    """
    Extracts all selectable text from each page of a PDF accessible via a URL, handling redirects.

    Args:
        url (str): The URL pointing to the PDF (or a redirect to the PDF).

    Returns:
        list: A list of strings, where each string contains the text from one page.
              If no text is found on a page, a message is included for that page.
              If an error occurs, a list with an error message is returned.
    """
    try:
        # Download the PDF from the URL, following redirects
        headers = {'User-Agent': 'Mozilla/5.0'}  # Add user-agent to avoid server blocks
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes

        # Check if the response content is a PDF
        content_type = response.headers.get('Content-Type', '')
        if 'application/pdf' not in content_type.lower():
            return ["Error: The URL does not point to a PDF file"]

        # Verify the content starts with %PDF to ensure it's a valid PDF
        if not response.content.startswith(b'%PDF'):
            return ["Error: The response content is not a valid PDF"]

        # Try extracting text with pdfplumber
        try:
            with warnings.catch_warnings():  # Suppress CropBox warnings
                warnings.simplefilter("ignore")
                with pdfplumber.open(io.BytesIO(response.content)) as pdf:
                    all_text = []
                    for page_number, page in enumerate(pdf.pages, start=1):
                        text = page.extract_text()
                        if text:
                            # Clean the text by removing extra whitespace and empty lines
                            cleaned_text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
                            all_text.append(f"Page {page_number}:\n{cleaned_text}")
                        else:
                            all_text.append(f"Page {page_number}: No selectable text found")
                    return all_text
        except Exception as e:
            # Fallback to PyPDF2 if pdfplumber fails
            try:
                pdf_reader = PyPDF2.PdfReader(io.BytesIO(response.content))
                all_text = []
                for page_number, page in enumerate(pdf_reader.pages, start=1):
                    text = page.extract_text()
                    if text:
                        cleaned_text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
                        all_text.append(f"Page {page_number}:\n{cleaned_text}")
                    else:
                        all_text.append(f"Page {page_number}: No selectable text found")
                return all_text
            except Exception as fallback_e:
                return [f"Error extracting text: pdfplumber failed with '{e}', PyPDF2 failed with '{fallback_e}'"]

    except requests.RequestException as e:
        return [f"Error downloading PDF: {e}"]
    except Exception as e:
        return [f"Error processing PDF: {e}"]

# Example usage
url = 'https://a836-edms.nyc.gov/dctm-rest/repositories/dofedmspts/StatementSearch?bbl=2023330031&stmtDate=20250215&stmtType=SOA'
extracted_content = extract_all_text_from_url(url)

# Print the extracted content for each page
for page_content in extracted_content:
    print(page_content)
    print('-' * 50)  # Separator between pages

Page 1:
84016142502150100140001 NYNP
Property Tax Bill Quarterly  Statement
Activity through February 15, 2025
Owner name: FAE HOLDINGS 395224R, LLC
Property address: 289MORRIS AVENUE
Amount Due 04/01/25: $0.00
#840161425021501#
FAE HOLDINGS 395224R, LLC
C/0 CACTUS PROPERTIES 4 LLC
129 BIRCH HILL RD.
LOCUST VALLEY NY11560-18411400.01 -ZB -40 -2 -0 -4 -1561
Borough: 2     Block: 02333     Lot:0031
Write this in your check's memo line: BBL 2-02333-0031
5536  20233300310  0000000000  250401  1  2025  8How much do I owe?
Outstanding charges $0.00
New charges $0.00
Total amount due by April 1, 2025 $0.00
Make checks payable & mail payment to:
NYC Department of Finance
PO Box 5536
Binghamton NY  13902-5536Borough
2Block
02333Lot
0031
--------------------------------------------------
Page 2:
February 15, 2025
Fae Holdings 395224R, LLC
289Morris Avenue
2-02333-0031
Page2
Billing Summary Amount
Outstanding charges
(Sum of unpaid balance and interest fees from billing periods)$0.00
New charges


In [4]:
# Read the data into a DataFrame
df = pd.read_csv("/content/subset_041125_update.csv")

# Display results
print(df[['BoroCode', 'Block', 'Lot', 'BBL']])

bbl_list = df['BBL'].to_list()

      BoroCode  Block  Lot         BBL
0            2   2260    1  2022600001
1            2   2260    4  2022600004
2            2   2260   34  2022600034
3            2   2261    3  2022610003
4            2   2277    1  2022770001
...        ...    ...  ...         ...
2170         4  15008    8  4150080008
2171         4  15008   33  4150080033
2172         4  15009   25  4150090025
2173         4  15009   51  4150090051
2174         4  15012    6  4150120006

[2175 rows x 4 columns]


In [7]:
# Base URL template
url_template = 'https://a836-edms.nyc.gov/dctm-rest/repositories/dofedmspts/StatementSearch?bbl={}&stmtDate=20250215&stmtType=SOA'

In [10]:
# Suppress all pdfminer warnings by redirecting its logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Set up logging for our application
logging.basicConfig(
    filename='bbl_processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def download_and_extract_text(url, max_retries=10, initial_delay=2):
    """
    Download a PDF from a URL and extract text lines from the first page, with retries on connection errors.

    Args:
        url (str): The URL of the PDF to download.
        max_retries (int): Maximum number of retry attempts (default: 3).
        initial_delay (int): Initial delay in seconds before retrying (default: 1).

    Returns:
        list: Extracted text lines if successful, or an error message string if failed.
    """
    delay = initial_delay
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    for attempt in range(max_retries):
        try:
            # Attempt to download the PDF
            response = requests.get(url, headers=headers, timeout=20)
            response.raise_for_status()

            # Verify the content is a PDF
            content_type = response.headers.get('Content-Type', '')
            if 'application/pdf' not in content_type.lower():
                return f"Error: URL does not point to a PDF file (Content-Type: {content_type})"

            # Verify content starts with %PDF
            if not response.content.startswith(b'%PDF'):
                return "Error: Response content is not a valid PDF"

            # Try extracting text with pdfplumber first
            try:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")  # Additional safety
                    pdf_file = io.BytesIO(response.content)
                    with pdfplumber.open(pdf_file) as pdf:
                        first_page = pdf.pages[0]
                        text = first_page.extract_text()
                        if not text:
                            return "Error: No text extracted from the PDF"
                        return text.split('\n')
            except Exception as e:
                # Fallback to pymupdf
                try:
                    pdf_file = io.BytesIO(response.content)
                    with pymupdf.open(stream=pdf_file, filetype="pdf") as doc:
                        first_page = doc[0]
                        text = first_page.get_text("text")
                        if not text:
                            return "Error: No text extracted from the PDF using pymupdf"
                        return text.split('\n')
                except Exception as fallback_e:
                    return f"Error extracting text: pdfplumber failed with '{e}', pymupdf failed with '{fallback_e}'"

        except requests.ConnectionError as e:
            if attempt < max_retries - 1:
                logging.warning(f"Connection error for {url}: {e}. Retrying in {delay}s...")
                time.sleep(delay)
                delay *= 5
            else:
                return f"Error downloading PDF after {max_retries} attempts: {e}"
        except requests.HTTPError as e:
            return f"HTTP Error: {e}"
        except requests.RequestException as e:
            return f"Error downloading PDF: {e}"

def extract_address(lines):
    """Extract the mailing address starting after the line with two '#' symbols."""
    hash_line_index = -1
    for i, line in enumerate(lines):
        if line.count('#') >= 2 and re.match(r'^#.*#$', line.strip()):
            hash_line_index = i
            break

    if hash_line_index == -1 or hash_line_index + 1 >= len(lines):
        return "Address not found: No line with two '#' symbols or insufficient lines follow"

    address_lines = []
    start_index = hash_line_index + 1

    if start_index < len(lines):
        line = lines[start_index].replace("Make checks payable & mail payment to:", "").strip()
        address_lines.append(line)

    if start_index + 1 < len(lines):
        line = lines[start_index + 1].replace("NYC Department of Finance", "").strip()
        address_lines.append(line)

    if start_index + 2 < len(lines):
        line = lines[start_index + 2].strip()
        address_lines.append(line)

    if start_index + 4 < len(lines):
        line = lines[start_index + 4].replace("Binghamton NY 13902-5536", "").strip()
        address_lines.append(line)

    return '\n'.join(address_lines)

# Dictionary to store results
results = {}

In [11]:
# Process each BBL with a clean progress bar
for bbl in tqdm(bbl_list[:5], desc="Processing BBLs", file=sys.stdout):
    try:
        url = url_template.format(bbl)
        logging.info(f"Processing BBL {bbl}: {url}")
        text_lines = download_and_extract_text(url, max_retries=3, initial_delay=1)
        if isinstance(text_lines, list):
            address = extract_address(text_lines)
            results[bbl] = address
            logging.info(f"Success for BBL {bbl}: {address}")
        else:
            results[bbl] = text_lines
            logging.warning(f"Failed for BBL {bbl}: {text_lines}")
        time.sleep(0.5)  # Rate limiting
    except Exception as e:
        error_msg = f"Unexpected error for BBL {bbl}: {str(e)}"
        results[bbl] = error_msg
        logging.error(error_msg)

# Save results to a file
with open('bbl_results.txt', 'w') as f:
    for bbl, address in results.items():
        f.write(f"BBL: {bbl}\nAddress:\n{address}\n{'-'*40}\n")

Processing BBLs: 100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


In [12]:
# join results with data and output a new csv

results_df = pd.DataFrame(list(results.items()), columns=['BBL', 'Address'])

df['BBL'] = df['BBL'].astype(str)
results_df['BBL'] = results_df['BBL'].astype(str)

merged_df = pd.merge(df, results_df, on='BBL', how='left')

merged_df['Address_y'] = merged_df['Address_y'].str.replace(
    r'\nBorough: \d+\s+Block: \d+\s+Lot: \d+$',
    '',
    regex=True
)

merged_df.to_csv("MapPluto_Subset_with_Mailing_Addr.csv")